## Importing the Libraries

## Loading Required Files

In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
!pip install tensorflow==2.2.1
from tensorflow.keras.models import load_model, model_from_json

ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.2.1


ModuleNotFoundError: No module named 'tensorflow.keras'

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
headphones = cv2.imread('./person.jpg', 0)
image = cv2.imread('human.jpg')
model = load_model('phone_model.h5')
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model1 = model_from_json(loaded_model_json)
# model1.load_weights("model.h5")

NameError: name 'load_model' is not defined

## Function to detect the face

In [ ]:
def face(frame, prevX, prevY, prevW, prevH, frame_no, dont):
    photo = 0 
    face_rects = face_cascade.detectMultiScale(frame, minNeighbors = 20)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255), 10)
        if abs(prevX - x) <= 2 and abs(prevY - y) <= 2 and abs(prevW - w) <= 2 and abs(prevH - h) <= 2:
            frame = cv2.putText(frame, 'Photo', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            photo += 4
            prevX = x
            prevY = y
            prevW = w
            prevH = h
        if frame_no == 1:
            prevX = x
            prevY = y
            prevW = w
            prevH = h
    if np.size(face_rects) > photo + 4 and not dont:
        frame = cv2.putText(frame, 'More than one face visible', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        dont = True
    elif np.size(face_rects) <= photo and not dont:
        frame = cv2.putText(frame, 'Face not Visible properly', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        dont = True
    return frame, prevX, prevY, prevW, prevH, dont

## Main function

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from tensorflow.keras.models import load_model, model_from_json
prevX = 0
prevY = 0
prevW = 0
prevH = 0
frame_no = 0
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    dont = False
    ret, frame = cap.read()
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    img = Image.fromarray(frame)
    size = (224, 224)
    img = ImageOps.fit(img, size, Image.ANTIALIAS)
    image_array = np.asarray(img)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    #print(prediction[0][0])
    if prediction[0][0] > 0.4:
        frame = cv2.putText(frame, 'Phone visible', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        dont = True
    try:
        frame1 = cv2.GaussianBlur(frame,(15,15),10)
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(headphones,None)
        kp2, des2 = orb.detectAndCompute(frame1,None)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1,des2)
        matches = sorted(matches, key = lambda x:x.distance)
        if np.shape(matches)[0] > 50 and not dont:
            frame = cv2.putText(frame, 'Headphones visible', (200, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            dont = True
    except:
        break
    frame_no += 1
    photo = 0
    frame[20:170, 20:120, :] = image
    frame, prevX, prevY, prevW, prevH, dont = face(frame, prevX, prevY, prevW, prevH, frame_no, dont)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()